In [308]:
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [309]:
response = requests.get("https://www.techradar.com/reviews/samsung-galaxy-s22-plus-review")

soup = BeautifulSoup(response.content, 'html.parser')

In [310]:
print(soup.find("h1").get_text(),
soup.find("h2").get_text(),
sep="\n")

Samsung Galaxy S22 Plus review
A middle child that’s destined to be forgotten



In [311]:
text = str(soup.find("span",class_="chunk rating"))
match = re.search(r'Rating: (\d+) out of (\d+) stars', text)
if match:
    rating = match.group(0)
    print(rating)

Rating: 3 out of 5 stars


In [323]:
soup.find("div", class_=re.compile("pretty-verdict__verdict")).findChild('p').get_text()

'Poco is back with another hot deal. The Poco X3 NFC comes with a few features you might expect of a pricier Android handset, such as decent gaming performance thanks to its Snapdragon G-series processor and a 120Hz screen, with excellent battery life sealing the deal. This is a great option for those on a budget.'

In [328]:
for i in soup.find_all("div", class_=re.compile("pretty-verdict__")):
    for j in i.childGenerator():
        #print(j)
        if j.name in ['p','h4', 'ul', 'li']:
            print(j.get_text())

Poco is back with another hot deal. The Poco X3 NFC comes with a few features you might expect of a pricier Android handset, such as decent gaming performance thanks to its Snapdragon G-series processor and a 120Hz screen, with excellent battery life sealing the deal. This is a great option for those on a budget.

Pros

+Strong gaming performance+Excellent battery life+Great 120Hz wide color gamut screen

Cons

-Struggling camera-Sometimes erratic backlight-It’s quite large, a little heavy


In [314]:
article_body = soup.find("div", id="article-body")

In [315]:
print(article_body.name)

div


In [316]:
article_body = soup.find("div", id="article-body")
first_h2 = article_body.find('h2')
second_h2 = first_h2.find_next('h2')
content = 'TWO-MINUTE REVIEW: \n'
for element in first_h2.next_siblings: # 遍历第一个h2标签的兄弟节点
    if element == second_h2: # 如果遍历到第二个h2标签，就退出循环
        break
    if element.name == "p":
        content+=element.get_text()+"\n"
    # if element.name: # 如果节点是标签
    #     content += str(element) # 将标签添加到内容中
    # else: # 如果节点是文本
    #     content += element.strip() # 将文本添加到内容中

print(content)

TWO-MINUTE REVIEW: 
Before getting to to the Samsung Galaxy S22 Plus it's worth noting that we now have a Samsung Galaxy S23 Plus review, as the S22 Plus's successor has been unveiled. You might also want to check out our Samsung Galaxy S23 review and Samsung Galaxy S23 Ultra review. So the S22 Plus is no longer one of Samsung's latest flagships. That doesn't mean it's not worth considering though.
That said, of the three members of Samsung’s 2022 flagship line, the Galaxy S22 Plus feels like the one that most people will forget about.
The Samsung Galaxy S22 is the one that’s easy to recommend: it’s small enough to fit comfortably in your hand, and has a low price that won’t immediately alienate buyers. And then there’s the S22 Ultra with its high price but its S Pen stylus, premium design and plentiful rear cameras which will interest those looking for a super-powerful handset.
Between those two phones, it’s harder to get excited for the Galaxy S22 Plus, the awkward middle child in Sa

In [317]:
import requests
from bs4 import BeautifulSoup
import re

ori_links = []

for i in tqdm(range(1,10)):
    response = requests.get("https://www.techradar.com/reviews/phones/mobile-phones/page/{}".format(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=re.compile(r'^https://www.techradar.com/reviews/')):
        ori_links.append(link.get('href'))
#print("\n".join([ i for i in links]))
links = []
for i in ori_links:
    if "https://www.techradar.com/reviews/phones/mobile-phones" not in i and "https://www.techradar.com/reviews/archive" not in i:
        links.append(i)



100%|██████████| 9/9 [00:03<00:00,  2.83it/s]


In [318]:
print("Totel Links",len(links))

Totel Links 90


In [352]:

print("Totel Links",len(links))

reviews = {}

for i in tqdm(links):
    response = requests.get(i)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find("h1").get_text().strip()
    subtitle = soup.find("h2").get_text().strip()

    match = re.search(r'Rating: (\d+) out of (\d+) stars', str(soup.find("span",class_="chunk rating")))
    if match:
        rating = match.group(0)
    else:
        rating = ""
    
    #summary = soup.find("div", class_=re.compile("pretty-verdict__verdict")).findChild('p').get_text()
    #pros_cons = "\n".join([ i.get_text().strip() for i in soup.find_all("div", class_=re.compile("pretty-verdict__"))[1:]])
    summary = ""
    for each_sum in soup.find_all("div", class_=re.compile("pretty-verdict__")):
        for each_p in each_sum.childGenerator():
            #print(j)
            if each_p.name in ['p','h4', 'ul', 'li']:
                summary+=each_p.get_text()+"\n"
    #print(summary)
    content = ""
    try:
        article_body = soup.find("div", id="article-body")
        first_h2 = article_body.find('h2')
        second_h2 = first_h2.find_next('h2')
        second_h3 = first_h2.find_next('h3')
        #content = 'TWO-MINUTE REVIEW: \n'
        for element in first_h2.next_siblings: # 遍历第一个h2标签的兄弟节点
            if element == second_h2 or element == second_h3: # 如果遍历到第二个h2标签，就退出循环
                break
            if element.name == "p":
                content+=element.get_text()+"\n"
            #content+=element.get_text()+"\n"
    except Exception as e:
        print(i)
    
    reviews[title+": "+subtitle] = {
        "rating": rating,
        "summary_pros_cons": summary,
        "review" : content
    }


print("success", len(reviews.keys()))
        

Totel Links 90


100%|██████████| 90/90 [00:14<00:00,  6.02it/s]

success 90


In [353]:
print(reviews.keys())

dict_keys(['Nanoleaf Essentials A19 E27 smart bulb review: affordable smart lighting: A smart-home essential to take on Philips Hue', 'Motorola Edge 40 Pro review: A better Pixel 7 Pro?: A confident flagship with very few flaws.', 'Xiaomi 13 review: a sleek and sophisticated iPhone imitator: The Xiaomi 13 makes (almost) all the right moves', 'Hands on: Oppo Find X6 Pro – Forget the oddball looks, this is a charging demon: For those who like their camera arrays big and their charging fast', "Hands on: Samsung Galaxy A54 review – more awesome?: Samsung's new mid-range champ?", "Oppo Find N2 Flip review: the Z Flip 4's foil: A new challenger steps into the fold", "Hands on: Xiaomi 13 Lite review – the Civi's new suit: A Civi by any other name...", 'Ulefone Power Armor 18T review: Not anything like the Power Armor 16 or 17 Pro', 'Unihertz Tank review: An oddly accurate name for a rugged phone', 'Hands on: Honor Magic 5 Pro review – a versatile flagship with a potentially world-beating came

In [354]:
#convert to json
import json
with open('techradar_phone_reviews.json', 'w') as f:
    json.dump(reviews, f)